In [2]:
# basic imports for connecting to coinbase pro and being able to execute trades
import pandas as pd
import cbpro
from auth_credentials import (api_secret, api_key, api_pass)

# print('test'); making sure everything imported correctly

WebsocketClient Class

In [3]:
class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url = 'wss://ws-feed-public.sandbox.exchange.coinbase.com'
        self.message_count = 0

    def on_message(self,msg):
        self.message_count += 1
        msg_type = msg.get('type',None)
        if msg_type == 'ticker':
            time_val = msg.get('time',('-'*27))
            price_val = msg.get('price',None)
            price_val = float(price_val) if price_val is not None else 'None'
            product_id = msg.get('product_id',None)

            print(f"{time_val:30} {price_val: .3f} {product_id}\tchannel type:{msg_type}")

    def on_close(self):
        print(f"<---Websocket connection closed--->\n\tTotal messages: {self.message_count}")

In [8]:
# stream = TextWebsocketClient(products=['BTC-USD','ETH-USD'],channels=['ticker'])
# stream.start()

# Trying to test out data stream; not entirely working as expected

---------------------------     16491.180 BTC-USD	channel type:ticker
---------------------------     0.000 ETH-USD	channel type:ticker
Connection is already closed. - data: None
<---Websocket connection closed--->
	Total messages: 2
Connection is already closed. - data: None
<---Websocket connection closed--->
	Total messages: 2
Connection is already closed. - data: None
<---Websocket connection closed--->
	Total messages: 3
Connection is already closed. - data: None
<---Websocket connection closed--->
	Total messages: 3


Client Auth

In [11]:
url = 'https://api-public.sandbox.exchange.coinbase.com'
client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url=url
)

Placing Market Orders


In [12]:
# testing with market order for now; will likely want to place limits and stops
# parameters: size => crypto amount, funds => usd
# fees will be .5% for limits and market orders under 10k notional value
# --- in the meantime, will need to account for that level of fees
# --- later, if ever sizing up enough, could implement more dynamic fee structure
# --- probably a good idea to create a dict with the fee structures for later use though
client.place_market_order(product_id='BTC-USD',side='buy',funds=50)
"""
returns/prints a dict as follows:
{'id': '53ecd854-4d01-431b-939a-bb6dfe6107bd',
 'product_id': 'BTC-USD',
 'side': 'buy',
 'stp': 'dc',
 'funds': '49.70178926',
 'specified_funds': '50',
 'type': 'market',
 'post_only': False,
 'created_at': '2022-12-06T19:03:42.985687Z',
 'fill_fees': '0',
 'filled_size': '0',
 'executed_value': '0',
 'status': 'pending',
 'settled': False}
"""

{'id': '53ecd854-4d01-431b-939a-bb6dfe6107bd',
 'product_id': 'BTC-USD',
 'side': 'buy',
 'stp': 'dc',
 'funds': '49.70178926',
 'specified_funds': '50',
 'type': 'market',
 'post_only': False,
 'created_at': '2022-12-06T19:03:42.985687Z',
 'fill_fees': '0',
 'filled_size': '0',
 'executed_value': '0',
 'status': 'pending',
 'settled': False}

In [ ]:
# let's start with just looking at a couple products (btc-usd, eth-usd, etc)
# calculate a couple basic metrics from looking at historical data, like 10,20,60, moving avgs
# can then look into some metrics like RSI to help better inform decisions
# want to develop some sort of triggers for when to enter a trade
# --- for instance, price near a support level, low-ish rsi, look to enter a buy
# --- after initializing a position, then can set a stop (say 2.5%)
# --- --- in current environment, may be a good idea to have a rolling stop
# --- --- until can improve logic for exiting, if we see say a +3% move from entry, could set stop to +2% from entry to try to at least make a small profit
# --- --- can always later adjust this, but for starters capital preservation is hugely important
# --- 
# --- if up say 5% from entry lowest entry, maybe look to reduce that position by 50%; could also move up stop on remainder?
# --- then, if up another 5%, fully exit that position for starters and look for new triggers
# ---
# --- if have reduced a position and retrace back to entry point, could then re-top off position or enter again for full amount
# --- will want some sort of logic that may dynamically adjust stops based on size of position and amount of scalps
# --- to start, should keep it very basic. keep notional size per trade to say $50 or max of 10% of total USD balance
# --- if only looking at a couple products, total capital deployed at any time should be minimal relative to USD balance
# --- once I have some metrics on performance, will look to continually make tweaks to trigger logic and sizing

# First iteration of algo will be 'dumb' and just look to make small scalps while minimizing losses
# --- first, basic logic will try to be tuned for trading when tokens are somewhat 'range-bound'
# --- --- to that end, may more heavily weight shorter-term SMAs for trigger decisions
# --- will want to continue to improve the trigger logic with other metrics
# --- will also want to have some sort of logic to 'kill' the algo
# --- --- for instance, if price breaches recent SMAs or recent support/resistances, should turn off until better logic in place
# --- --- additionally, will want to keep track of some sort of momentum in either direction to modify trigger decisions

# For simplicity's sake, will just look at token-usd pairs to start
# Later, want to try to add in logic for looking at token-token pairs and metrics; could be an indicator for which are better to buy/sell at a given time



In [ ]:
# For initial implementation, first start by using the public client in cbpro
# write out the basic logic for what kind of triggers to look for
# if a trigger is met to make a buy, maybe just print that out
# then, can keep track of the 'buy' order by adding to an array or dict or pd.df
# --- for this testing method, will want to keep track of 'mental' stops from opening price
# can then check the feed every few minutes or so to see whether or not price has breached stop or not
# --- if yes, then could print out that we are selling for a loss
# conversely, if price is up >2.5% from entry, could set new 'stop' threshold to say 1.5%
# --- from there, that will be the reference point to check on every few minutes
# --- in actual implementation, will want to instead figure out how to update stop orders
# if price is up more from that point, can then continue to roll up the stop price
# if price >5% from entry, then for now can look to 'close' out the trade

# should keep track of a ledger of all trades by id or something
# --- once an opening trade is closed out, add to the list with PnL associated

# once that is in place, can look to implement that basic logic in live mode
# --- will need to modify how storing/keeping track of actual token balances

# will want some safety triggers in place to shut the algo off if certain events take place
# but from there, can continue to build out better trading logic, sizing, etc


Account History


In [18]:
accounts = client.get_accounts()

for acc in accounts:
    currency = acc.get('currency')
    if currency == 'BTC':
        acc_id = acc.get('id')
        # print(acc_id)

3f664e68-3ab2-47ad-a291-37a02e69b9ee


In [21]:
btc_history = client.get_account_history(acc_id)

In [22]:
import json

# shows trade history in the following format:
"""
{
 "id": "421436199",
 "amount": "0.0050185700000000",
 "balance": "0.0050185700000000",
 "created_at": "2022-12-06T19:03:43.019582Z",
 "type": "match",
 "details": {
  "order_id": "53ecd854-4d01-431b-939a-bb6dfe6107bd",
  "product_id": "BTC-USD",
  "trade_id": "39717525"
 }
"""
# amount is the order amount; balance is the resulting balance in the account
for hist in btc_history:
    print(json.dumps(hist,indent=1))

{
 "id": "421436199",
 "amount": "0.0050185700000000",
 "balance": "0.0050185700000000",
 "created_at": "2022-12-06T19:03:43.019582Z",
 "type": "match",
 "details": {
  "order_id": "53ecd854-4d01-431b-939a-bb6dfe6107bd",
  "product_id": "BTC-USD",
  "trade_id": "39717525"
 }
}


Doing some Public Client stuff first to test out basic logic


In [27]:
pub_client = cbpro.PublicClient()

# gets list of all available products
data = pd.DataFrame(pub_client.get_products())
# print(len(data)) # 576 total products; want to just focus on a couple for now
# for simplicity's sake, just look at ETH-USD and BTC-USD; can add others later

# consider just pulling previous year of data to start
from datetime import datetime
from dateutil.relativedelta import relativedelta

today = datetime.now()
year_ago = today - relativedelta(years=1)
six_month_ago = today - relativedelta(months=3)
iso_today = today.isoformat()
iso_year_ago = year_ago.isoformat()
iso_6m_ago = six_month_ago.isoformat()
print(iso_today)
print(iso_year_ago)
print(iso_6m_ago)

# pulling historical eth data
# NOTE: get_product_historic_rates output limited to 200 rows, otherwise error is thrown
# --- if want to get more data from a period of time, will likely need to loop over and concat the pd.DataFrame
# --- for now, just look at some basic data using this simple daily, 3month sample; not ideal for a more robust strategy though
historical_eth = pd.DataFrame(pub_client.get_product_historic_rates(product_id='ETH-USD',start=iso_6m_ago,end=iso_today,granularity=86400))
historical_eth.columns= ["Date","Open","High","Low","Close","Volume"]
historical_eth['Date'] = pd.to_datetime(historical_eth['Date'], unit='s')
historical_eth.set_index('Date', inplace=True)
historical_eth.sort_values(by='Date', ascending=True, inplace=True)

# print(historical_eth.head())

# going to also pull last day's stats in 15m increments; will try to compare this to previous month metrics in some manner
last_day_eth = pd.DataFrame(pub_client.get_product_historic_rates(product_id='ETH-USD',start=(today-relativedelta(days=1)).isoformat(),end=iso_today,granularity=900))
last_day_eth.columns= ["Date","Open","High","Low","Close","Volume"]
last_day_eth['Date'] = pd.to_datetime(last_day_eth['Date'], unit='s')
last_day_eth.set_index('Date', inplace=True)
last_day_eth.sort_values(by='Date', ascending=True, inplace=True)

# print(last_day_eth.tail())
# print(last_day_eth.head())
# Things to consider: previous day's range, previous day's VWAP (or something close), maybe RSI as well

# Compare that against the 6month (and ideally also 1m and 1y back, later) data
# Try to think of some basic triggers to make decisions on
# Eventually, will want to store this data in some sort of class that gets built out later
# --- should have some functions that would regenerate the historical data on a rolling basis
# --- maybe the daily gets regenerated every 4hrs or so, longer timeframes every day


# From here, can look to build out some basic metrics for this time frame
# With 6month data, can very simply look at the 5d, 10d, 20d moving averages
# Those can be the starting indicators to compare against; later can add more granularity and other stuff

# For the daily, can look into some smaller time frame averages; 4hr, 12hr, 24hr
# If either of those metrics near one of the others, maybe that could be a simple trigger?

# Also, at a later implementation, I want to factor in the realized volatility
# That could be useful indicator for either expanding some stop losses/profit targets, or vice versa

historical_eth['5 SMA'] = historical_eth.Close.rolling(5).mean()
historical_eth['10 SMA'] = historical_eth.Close.rolling(10).mean()
historical_eth['20 SMA'] = historical_eth.Close.rolling(20).mean()
historical_eth['50 SMA'] = historical_eth.Close.rolling(50).mean()
print(historical_eth.tail())
# initial thoughts: see that 50SMA > than 20SMA by ~100 bucks; could maybe set those to initial resistances/support for first iteration
# --- obviously not a strong idea long term, but could implement something saying that while that gap exists larger than a certain magnitude, could treat that as a target range
# --- once the gap starts to converge more, than don't view it as such anymore

last_day_eth['4hr SMA'] = last_day_eth.Close.rolling(16).mean()
last_day_eth['8hr SMA'] = last_day_eth.Close.rolling(32).mean()
last_day_eth['12hr SMA'] = last_day_eth.Close.rolling(48).mean()
last_day_eth.tail()

# can see that the 4hr approaching the 20 SMA; if it gets close to touching, could look to make a buy
# then, either implement a stop for it, or could try to have 1 double down if it dips 2.5% lower
# --- after that, create a stop for the balance

2022-12-08T10:24:18.451844
2021-12-08T10:24:18.451844
2022-09-08T10:24:18.451844
               Open     High      Low    Close         Volume     5 SMA  \
Date                                                                      
2022-12-04  1240.27  1287.44  1240.91  1279.68  195680.277320  1277.494   
2022-12-05  1246.73  1305.75  1279.67  1259.23  334071.846813  1270.386   
2022-12-06  1241.83  1274.57  1259.32  1271.19  310591.432758  1269.328   
2022-12-07  1215.00  1277.49  1271.36  1231.56  294892.065257  1256.514   
2022-12-08  1221.72  1253.84  1231.49  1250.41  179342.898019  1258.414   

              10 SMA     20 SMA     50 SMA  
Date                                        
2022-12-04  1236.738  1211.5655  1334.2810  
2022-12-05  1242.795  1211.9400  1333.3448  
2022-12-06  1249.434  1214.7390  1332.1388  
2022-12-07  1253.260  1216.3450  1330.5544  
2022-12-08  1261.577  1218.3190  1329.8674  


,Open,High,Low,Close,Volume,4hr SMA,8hr SMA,12hr SMA
Date,,,,,,,,
2022-12-08 09:15:00,1231.09,1233.58,1232.96,1231.34,1278.245012,1230.975000,1230.380312,1230.484375
2022-12-08 09:30:00,1229.28,1233.72,1231.40,1229.41,2395.146840,1231.138750,1230.552188,1230.461458
2022-12-08 09:45:00,1229.01,1231.07,1229.36,1229.93,1401.174443,1231.204375,1230.640625,1230.408958
2022-12-08 10:00:00,1229.86,1232.10,1229.86,1231.22,1426.790073,1231.492500,1230.785312,1230.457708
2022-12-08 10:15:00,1230.78,1234.77,1231.22,1233.09,1823.838876,1231.778125,1230.925625,1230.562917


Public Client class w/ basic trading logic


In [111]:
import time

class PublicWebsocketClient(cbpro.PublicClient):
    def __init__(self,product='ETH-USD'):
        super().__init__()
        self.product = product
        self._set_historical_6mo()
        self._set_past24hr()
        self.open_trades = [] # in live class, will want to build this out by pulling trades from account history
        # --- for now, will likely want to have this be an array of dicts with open price, stop price, notional size, etc
        # self._set_update_function()
        self.capital_available = 500 # will pull the USD balance for the real client
        self.pnl = []

    # functions for setting past 6mo daily info and past24hr 15min info and building out SMAs
    def _set_historical_6mo(self):
        today = datetime.now()
        six_month_ago = today - relativedelta(months=6)
        self.historical = pd.DataFrame(self.get_product_historic_rates(product_id=self.product,start=six_month_ago.isoformat(),end=today.isoformat(),granularity=86400))
        self.historical.columns= ["Date","Open","High","Low","Close","Volume"]
        self.historical['Date'] = pd.to_datetime(self.historical['Date'], unit='s')
        self.historical.set_index('Date', inplace=True)
        self.historical.sort_values(by='Date', ascending=True, inplace=True)
        # building out some SMAs
        self.historical['5 SMA'] = self.historical.Close.rolling(5).mean()
        self.historical['10 SMA'] = self.historical.Close.rolling(10).mean()
        self.historical['20 SMA'] = self.historical.Close.rolling(20).mean()
        self.historical['50 SMA'] = self.historical.Close.rolling(50).mean()
        

    def _set_past24hr(self):
        today = datetime.now()
        yesterday = today - relativedelta(days=1)
        self.past24hr = pd.DataFrame(self.get_product_historic_rates(product_id=self.product,start=yesterday.isoformat(),end=today.isoformat(),granularity=900))
        self.past24hr.columns= ["Date","Open","High","Low","Close","Volume"]
        self.past24hr['Date'] = pd.to_datetime(self.past24hr['Date'], unit='s')
        self.past24hr.set_index('Date', inplace=True)
        self.past24hr.sort_values(by='Date', ascending=True, inplace=True)
        
        self.past24hr['4hr SMA'] = self.past24hr.Close.rolling(16).mean()
        self.past24hr['8hr SMA'] = self.past24hr.Close.rolling(32).mean()
        self.past24hr['12hr SMA'] = self.past24hr.Close.rolling(48).mean()

    # will likely need a function that turns the algo 'on'
    # when on, the algo will periodically check (maybe every 15 minutes) and see if a trigger is met to either open or close a trade
    # --- so, will need to learn how to implement keeping it on and checking periodically
    # --- will also need to figure out how to be able to shut it off if need be

    def check_buy_trigger(self,bid_ask=None):
        # should pass in bid_ask later; also, if ever sizing up enough, may need to have a deeper order book instead of default level=1
        # bid_ask = self.get_product_order_book(product_id=self.product)

        if bid_ask == None:
            return 'No bid ask entered'

        ask = float(bid_ask['asks'][0][0]) # just gets price or orderbook data; for now, assuming notional value small enough to get filled
        # print(ask)

        support = float(self.historical['20 SMA'][-1:].values[0]) # hacky way to get latest 20 SMA which, for now, going to use as support metric
        # print(support)

        resistance = float(self.historical['50 SMA'][-1:].values[0]) # again, for now; will want to make trigger logic more dynamic down the road

        print(f'Checking for buy trigger. Support: {support}, Resistance: {resistance}, Current ask: {ask}')
        
        if ask < support:
            print('should make a buy; currently under support')
            self._make_buy(price=(ask+1)) # setting limit order for now above ask; will likely want to do something similar in live
        elif (ask-support)/support < .025:
            print('within 2.5 pct of support line; close to making a buy')
        elif (ask-resistance)/resistance >= .025:
            print('above resistance by 2.5 pct; making a buy in case of breakout')
            self._make_buy(price=(ask+1))
        else:
            print('not time to make a buy; greater than 2.5 pct away from support and not more than 2.5 pct above resistance')

    def _make_buy(self,price,notional=50):
        # assuming will be paying the .5% fees for making/taking for now
        buy_dict = {'price': price, 'usd_amt': notional, 'token_amt': (notional*.995)/price, 'stop': price*.975, 'fees': notional*.005}
        self.open_trades.append(buy_dict)
        self.capital_available -= notional
        print(f'Entering into new buy position: {buy_dict}\n Available capital: {self.capital_available}')

    def check_sale_trigger(self,bid_ask=None):
        # assuming that this function only called if open_trades not empty
        if len(self.open_trades) == 0:
            return 'No open trades'

        if bid_ask == None:
            return 'No bid ask entered'

        open_trade = self.open_trades[0] # for now, limiting position to single open trade for single product; later, could have multiple trades for multiple products

        bid = float(bid_ask['bids'][0][0])

        print(f'checking for sell trigger: Entry price: {open_trade["price"]}, Stop: {open_trade["stop"]}, Current bid: {bid}')

        if bid < open_trade['stop']:
            print('Stopped out of trade; closing for loss')
            self._make_sale(sell_price=(bid-1),trade=open_trade)

        elif bid >= 1.05*open_trade['price']:
            print('Closing out trade for 5 pct or greater gain')
            self._make_sale(sell_price=(bid-1),trade=open_trade)
        
        # some basic stop update logic here
        elif bid >= 1.025*open_trade['price']:
            pct_above = (bid - open_trade['price'])/open_trade['price']

            # in theory, basic logic increases stop price to min 1.5% higher than entry up to just under 4% above entry
            # less fees, would aim to lock in small profit/scratch at worst

            new_stop = open_trade['price'] * (1+ pct_above - .01)
            if new_stop > open_trade['stop']:
                old_stop = open_trade['stop']
                open_trade['stop'] = new_stop
                print(f'Increasing stop from {old_stop} to {new_stop}')
            else:
                print('Keeping position open; between target price and updated stop')

    def _make_sale(self, sell_price, trade):
        # need to send sell order to exchange here, likely with price a little less than the current bid
        usd = (sell_price*trade['token_amt'])*.995 # factoring in the .5% fee again here
        self.capital_available += usd
        if usd < trade['usd_amt']:
            print(f'sold for loss of {trade["usd_amt"]-usd} dollars, including fees')
        else:
            print(f'sold for gain of {usd-trade["usd_amt"]} dollars, including fees')

        self.open_trades.remove(trade)
        trade_summary = {'net_pnl': usd-trade["usd_amt"], 'net_pct_pnl': (usd-trade["usd_amt"])/trade["usd_amt"], 'capital_balance': self.capital_available}
        self.pnl.append(trade_summary)
        print(f"Trade summary: {trade_summary}")

    def test_mock_buy(self,price):
        if len(self.open_trades) ==0:
            self._make_buy(price=price)

    # todo: write a tick() function that grabs the current orderbook, then runs the buy/sale trigger checks
    # --- for now, with simple logic, only do check_buy if no open trades, check_sell if is open trade
    # --- will need to refactor above functions a bit after the tick function in place
    # --- also, will need to refactor again later to accomodate for multiple open trades potentially across multiple products

    # but for now, for simplicity sake, sticking with a single open position will be easiest to get up and running

    # for now, by default, will run every 5 minutes
    # also, will have a cutoff threshold in time to stop running
    # --- a sort of safety feature until I flesh out better trading trigger logic and how/when to stop running
    def tick(self,increment=300,total_ticks=12):
        tick_count=0
        while tick_count < total_ticks:
            # first, fetch the current bid/ask
            print(f'Tick iteration {tick_count+1}')
            bid_ask = self.get_product_order_book(product_id=self.product)

            # for now, only checking if should make buy if no open trade
            if len(self.open_trades) == 0:
                self.check_buy_trigger(bid_ask=bid_ask)
            else:
                self.check_sale_trigger(bid_ask=bid_ask)

            tick_count += 1
            print(f'\t Now sleeping for {increment} seconds...')
            time.sleep(increment)

            # should also have some sort of logic to re-pull historical and past24hr data
            # could simply just be checking whether increment*tick_count greater than a certain threshold
            # but then, if this were running for multiple days, would need to keep track of that threshold in some way

    

Testing Public class init


In [112]:
test_public = PublicWebsocketClient()


Testing functions from public class


In [113]:
# print(test_public.historical.tail())
# print(test_public.past24hr.tail())
# print(test_public.check_buy_trigger())
# print(test_public.check_sale_trigger())

test_public.test_mock_buy(1240)

test_public.tick(increment=30, total_ticks=100)

Entering into new buy position: {'price': 1240, 'usd_amt': 50, 'token_amt': 0.040120967741935486, 'stop': 1209.0, 'fees': 0.25}
 Available capital: 450
Tick iteration 1
checking for sell trigger: Entry price: 1240, Stop: 1209.0, Current bid: 1284.91
Increasing stop above entry price
	 Now sleeping for 30 seconds...
Tick iteration 2
checking for sell trigger: Entry price: 1240, Stop: 1272.51, Current bid: 1285.32
Increasing stop above entry price
	 Now sleeping for 30 seconds...
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/lukemultanen/.pyenv/versions/3.9.15/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/dp/f04slrm12hz3ycfx07_hr0tw0000gn/T/ipykernel_32411/205091694.py", line 8, in <module>
    test_public.tick(increment=30, total_ticks=100)
  File "/var/folders/dp/f04slrm12hz3ycfx07_hr0tw0000gn/T/ipykernel_32411/4097431803.py", line 158, in tick
    time.sleep(increment)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/lukemultanen/.pyenv/versions/3.9.15/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2052, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/lukemultanen/.pyenv/versions/3.9.15/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return FormattedTB.st